<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Reference-docs" data-toc-modified-id="Reference-docs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Reference docs</a></span></li><li><span><a href="#Start-up" data-toc-modified-id="Start-up-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Start up</a></span></li><li><span><a href="#Sign-representations" data-toc-modified-id="Sign-representations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sign representations</a></span></li><li><span><a href="#Object-similarity" data-toc-modified-id="Object-similarity-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Object similarity</a></span><ul class="toc-item"><li><span><a href="#Storing-similarity-computations" data-toc-modified-id="Storing-similarity-computations-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Storing similarity computations</a></span></li></ul></li><li><span><a href="#Clustering" data-toc-modified-id="Clustering-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Clustering</a></span><ul class="toc-item"><li><span><a href="#Algorithm" data-toc-modified-id="Algorithm-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Algorithm</a></span></li><li><span><a href="#A-few-runs" data-toc-modified-id="A-few-runs-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>A few runs</a></span></li></ul></li><li><span><a href="#Cluster-evaluation" data-toc-modified-id="Cluster-evaluation-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Cluster evaluation</a></span><ul class="toc-item"><li><span><a href="#Criteria" data-toc-modified-id="Criteria-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Criteria</a></span></li><li><span><a href="#Threshold-search" data-toc-modified-id="Threshold-search-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Threshold search</a></span></li></ul></li></ul></div>

<img align="left" src="images/P005381-obverse-photo.png" width="15%"/>
<img align="left" src="images/P005381-obverse-lineart-annot.png" width="15%"/>
<img align="right" src="images/P005381-reverse-photo.png" width="15%"/>
<img align="right" src="images/P005381-reverse-lineart.png" width="15%"/>

<p>
```
&P005381 = MSVO 3, 70
```
</p>
<p>
<img src="images/P005381-obverse-atf.png" width="40%"/>
<img src="images/P005381-reverse-atf.png" width="40%"/>
</p>

<img align="right" src="images/tf-small.png"/>


# Clustering

We want to get insights in the co-occurrences of signs on tablets in the 
[Uruk III/IV](http://cdli.ox.ac.uk/wiki/doku.php?id=proto-cuneiform)
corpus (4000-3100 BC).
These tablets have a poor archival context, since they come from rubbish pits, and may have been transported
from various different places than where they have been excavated.

In order to get more information about their chronology and context, we need to study the evolution of
the signs on the tablets. Clustering and collocation are prerequisites to do so.

In [collocation](collocation.ipynb) we did first steps in pairing signs
that co-occur on the same tablets, faces, columns, lines.

Now we want to *cluster* tablets based on the signs they contain.

**N.B.:** This notebook does not use the results from the collocation notebook.

The method borrows insights from finding parallel passages in the Hebrew Bible,
as executed in the [parallels notebook](https://github.com/ETCBC/parallels/blob/master/programs/parallels.ipynb).

## Data

We have downloaded the transcriptions from the 
**Cuneiform Digital Library Initiative**
[CDLI](https://cdli.ucla.edu),
and converted them to
[Text-Fabric](https://github.com/Dans-labs/text-fabric).
Read more about the details of the conversion in the
[checks](checks.ipynb) notebook.
For an introduction to Text-Fabric, follow the
[start](start.ipynb) tutorial.

## Reference docs
The functions used by this notebook are documented in the following places:

[Feature docs](https://github.com/Dans-labs/Nino-cunei/blob/master/docs/transcription.md)

[Cunei API](https://github.com/Dans-labs/Nino-cunei/blob/master/docs/cunei.md)

[Utils API](https://github.com/Dans-labs/Nino-cunei/blob/master/docs/utils.md)

[Text-Fabric API](https://github.com/Dans-labs/text-fabric)


# Authors

J. Cale Johnson and Dirk Roorda (see the 
[README](https://github.com/Dans-labs/Nino-cunei)
of this repository).

## Start up

We import the Python modules we need.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os, collections
from IPython.display import Markdown, display
from tf.fabric import Fabric

We set up our working locations on the file system.

In [3]:
GITHUB = 'https://github.com'
REPO_REL = 'Dans-labs/Nino-cunei'
REPO = f'~/github/{REPO_REL}'
SOURCE = 'uruk'
VERSION = '0.1'
CORPUS = f'{REPO}/tf/{SOURCE}/{VERSION}'
SOURCE_DIR = os.path.expanduser(f'{REPO}/sources/cdli')
PROGRAM_DIR = os.path.expanduser(f'{REPO}/programs')
TEMP_DIR = os.path.expanduser(f'{REPO}/_temp')
REPORT_DIR = os.path.expanduser(f'{REPO}/reports')
RESULT_DIR = f'{REPORT_DIR}/clustering'
RESULT_GH = f'{GITHUB}/{REPO_REL}/blob/master/reports/clustering'
TEMP_RESULT_DIR = f'{TEMP_DIR}/clustering'

We create the temporary and report directories, if they do not exist already.

In [4]:
sys.path.append(PROGRAM_DIR)
from cunei import Cunei
from utils import Compare

In [5]:
for cdir in (TEMP_DIR, REPORT_DIR, RESULT_DIR, TEMP_RESULT_DIR):
    os.makedirs(cdir, exist_ok=True)

In [6]:
TF = Fabric(locations=[CORPUS], modules=[''], silent=False )

This is Text-Fabric 3.2.2
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

32 features found and 0 ignored


In [7]:
api = TF.load('''
    grapheme prime repeat
    variant variantOuter
    modifier modifierInner modifierFirst
    damage uncertain remarkable written
    period name type identifier catalogId
    number fullNumber origNumber badNumbering
    crossref text
    srcLn srcLnNum
    op sub comments''')
api.makeAvailableIn(globals())
CUNEI = Cunei(api)
COMP = Compare(api, SOURCE_DIR, TEMP_DIR)

  0.00s loading features ...
   |     0.00s B catalogId            from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.02s B fullNumber           from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.02s B number               from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.06s B grapheme             from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.05s B srcLn                from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.02s B srcLnNum             from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B prime                from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.01s B repeat               from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.01s B variant              from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B variantOuter         from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B modi

We pick up where we left off in the [start](start.ipynb) tutorial: computing co-occurrences
by tablet. But we make the move to put our recipes into functions, that we will re-use and refine later on.

## Sign representations

We pre-compute the sets of distinct signs per object.

We also make an sign-index of the objects.

Objects without signs (after filtering out non-signs), will be excluded.

In [10]:
NA = {'', '…', 'X'}

def getSigns(nodeType):
    signsFromObject = collections.defaultdict(set)
    objectsFromSign = collections.defaultdict(set)
    emptyObjects = set()

    for obj in F.otype.s(nodeType):
        theseSigns = {
            CUNEI.atfFromSign(s)
            for s in L.d(obj, otype='sign')
            if F.grapheme.v(s) not in NA
        }
        for signRep in theseSigns:
            signsFromObject[obj].add(signRep)
            objectsFromSign[signRep].add(obj)
        if len(theseSigns) == 0:
            emptyObjects.add(obj)
    print(
        f'computed {len(objectsFromSign)} distinct sign representations'
        f' from {len(signsFromObject)} {nodeType}s'
        f' ({len(emptyObjects)} empty {nodeType}s)'
    )
    return (signsFromObject, objectsFromSign, emptyObjects)

First we will work with tablets.

In [11]:
(signsFromObject, objectsFromSign, emptyObjects) = getSigns('tablet')

computed 1526 distinct sign representations from 5674 tablets (722 empty tablets)


We show one of the empty tablets.

In [13]:
emptyTablet = sorted(emptyObjects)[100]
print('\n'.join(COMP.getSource(emptyTablet)))

&P005057 = ATU 6, pl. 91, VAT 21520
#version: 0.1
#atf: lang qpc
@obverse
@column 1
1. [...] , X [...]


![empty](images/P005057-photo.png)

## Object similarity

We will group the objects into clusters based on a concept of *similarity* between objects.

A handy measure is 

$$|T_1 \cap T_2| \over |T_1 \cup T_2|$$

In words: we take the number of signs in the intersection of the sign sets of two objects
and divide it by the number of signs in the union of the sign sets of those two objects.

It is the ratio between the amount of signs that the objects have in common and the 
total amount of signs they have.

If the two objects are equal, or even if they have the same sign set, their similarity is maximal, 
i.e. `1`.

If two objects have no signs in common, their similarity is minimal, i.e. `0`.

If the sign set of one object is contained in that of another object, the similarity is the
size of the smaller sign set divided by the bigger sign set.

In [14]:
def signSimilarity(t1, t2):
    signSet1 = signsFromObject[t1]
    signSet2 = signsFromObject[t2]
    return len(signSet1 & signSet2) / len(signSet1 | signSet2)

### Storing similarity computations

We are going to need many similarities between tablets, sometimes multiple times per pair.
That's why we are going to memoize the results of the similarity function.

In [15]:
simCache = {}
stats = [0, 0]

def memoSim(t1, t2):
    pair = (t1, t2) if t1 < t2 else (t2, t1)
    sim = simCache.get(pair, None)
    if sim is None:
        sim = signSimilarity(*pair)
        simCache[pair] = sim
        stats[1] += 1
    else:
        stats[0] += 1
    return sim

We need to see an example.
Here is the source code of two (admittedly handy chosen) tablets.

We then show their similarity.

In [16]:
tablet1 = T.nodeFromSection(('P001059',))
tablet2 = T.nodeFromSection(('P448701',))
source1 = '\n'.join(COMP.getSource(tablet1))
source2 = '\n'.join(COMP.getSource(tablet2))
signs1 = signsFromObject[tablet1]
signs2 = signsFromObject[tablet2]
intersection = signs1 & signs2
union = signs1 | signs2
lengthI = len(intersection)
lengthU = len(union)

print(source1)
print('\n - - - - - - - - - - - - \n')
print(source2)
print('\no-o-o-o-o-o-o-o-o-o-o-o-o\n')

print(' '.join(sorted(signs1)))
print('\n - - - - - - - - - - - - \n')
print(' '.join(sorted(signs2)))
print('\no-o-o-o-o-o-o-o-o-o-o-o-o\n')

print(f'intersection: length {lengthI}: {" ".join(sorted(intersection))} ')
print(f'union       : length {lengthU}: {" ".join(sorted(union))}')

print('\no-o-o-o-o-o-o-o-o-o-o-o-o\n')

print(f'\nSIMILARITY {memoSim(tablet1, tablet2)}')

&P001059 = ATU 5, pl. 039, W 9168,d
#version: 0.1
#atf: lang qpc
@obverse
@column 1
1. 2(N01) , APIN~a SZAM2#
2. 2(N14)# [...] , |KA2xLAM|#
3. [...] 2(N01)# , KINGAL#
4. [...] , [...]

 - - - - - - - - - - - - 

&P448701 = www archaeo-auction 003 
#atf: lang qpc 
@obverse 
@column 1 
1. 1(N46) 2(N19) 4(N41) , 
2. , AB~a APIN~a NUN~a X X 
@column 2 
1. , X X
2. , SZE~a DU NUN~a  
@reverse 
$ (not imaged) 

o-o-o-o-o-o-o-o-o-o-o-o-o

2(N01) 2(N14) APIN~a KA2 KINGAL LAM SZAM2

 - - - - - - - - - - - - 

1(N46) 2(N19) 4(N41) AB~a APIN~a DU NUN~a SZE~a

o-o-o-o-o-o-o-o-o-o-o-o-o

intersection: length 1: APIN~a 
union       : length 14: 1(N46) 2(N01) 2(N14) 2(N19) 4(N41) AB~a APIN~a DU KA2 KINGAL LAM NUN~a SZAM2 SZE~a

o-o-o-o-o-o-o-o-o-o-o-o-o


SIMILARITY 0.07142857142857142


## Clustering

We borrow from the concept of *cliques* and the method of obtaining
them demonstrated in the
[parallels notebook](http://localhost:8888/notebooks/etcbc/parallels/programs/parallels.ipynb#4.3-Cliques).

### Algorithm
The idea is to visit all objects, one by one, and form clusters as we go.

Suppose we have already a bunch of clusters and we visit a new object.

We determine the clusters that are close enough to the object.

If there are such clusters, we merge them and add the object to the new cluster.

If not, we make a new empty cluster and put the object in it.

We set a threshold to define what is "close enough".

In [43]:
progress = 1000000

def clustering(objects, threshold):
    print(f'clustering {len(objects)} objects with threshold {threshold}')
    i = 0
    j = 0
    clustersUnsorted = []
    for (o, obj) in enumerate(objects):
        added = None
        removable = set()
        for (k, cluster) in enumerate(clustersUnsorted):
            origCluster = tuple(cluster)
            for objC in origCluster:            
                if j >= progress:
                    print(
                        f'\t{o:>4} objects - {i:>9} x memoSim'
                        f' - {stats[0]:>8} lookups {stats[1]:>9} comps'
                    )
                    j = 0
                sim = memoSim(obj, objC)
                i += 1
                j += 1
                if sim >= threshold:
                    if added == None:    
                        # obj has not been added to any cluster yet
                        cluster.add(obj)
                        added = k        
                        # remember that we added obj to this cluster
                    else:                
                        # obj has alreay been added to another cluster:
                        # we merge this cluster with that one
                        clustersUnsorted[added] |= cluster
                        removable.add(k) 
                        # we remember that we have merged this cluster into another one,
                        # so we can throw away this cluster later 
                    break
        if added == None:
            clustersUnsorted.append({obj})
        else:
            if len(removable):
                clustersUnsorted = [
                    cluster
                    for (k,cluster) in enumerate(clustersUnsorted)
                    if k not in removable
            ]
    result = sorted(
        [
            tuple(sorted(cluster)) 
            for cluster in clustersUnsorted
        ]
    )
    print(
        f'\t{len(objects):>4} objects - {i:>9} x memoSim'
        f' - {stats[0]:>8} lookups {stats[1]:>9} comps'
    )

    print(f'{len(result)} clusters with threshold {threshold}')
    
    return result

### A few runs

We just run the clustering with a similarity threshold of `0.8` to see what happens.

In [44]:
objects = sorted(signsFromObject)

In [45]:
clusters = clustering(objects, 0.8)

clustering 5674 objects with threshold 0.8
	1414 objects -   1000000 x memoSim - 111248734 lookups  16090051 comps
	2000 objects -   2000000 x memoSim - 112248734 lookups  16090051 comps
	2450 objects -   3000000 x memoSim - 113248734 lookups  16090051 comps
	2829 objects -   4000000 x memoSim - 114248734 lookups  16090051 comps
	3163 objects -   5000000 x memoSim - 115248734 lookups  16090051 comps
	3465 objects -   6000000 x memoSim - 116248734 lookups  16090051 comps
	3742 objects -   7000000 x memoSim - 117248734 lookups  16090051 comps
	4000 objects -   8000000 x memoSim - 118248734 lookups  16090051 comps
	4243 objects -   9000000 x memoSim - 119248734 lookups  16090051 comps
	4473 objects -  10000000 x memoSim - 120248734 lookups  16090051 comps
	4691 objects -  11000000 x memoSim - 121248734 lookups  16090051 comps
	4900 objects -  12000000 x memoSim - 122248734 lookups  16090051 comps
	5100 objects -  13000000 x memoSim - 123248734 lookups  16090051 comps
	5292 objects -  1400

Only a few hundred objects got clustered with an other one.
Such a rare clustering is not very informative.
The threshold is too strict.

What if we work with a very generous threshold, say `0.1`?

In [46]:
clusters = clustering(objects, 0.1)

clustering 5674 objects with threshold 0.1
	5674 objects -    212638 x memoSim - 126551290 lookups  16090051 comps
9 clusters with threshold 0.1


Note that the caching of the similarity computation pays off: looking up a value
is quicker than computing it.

We now have very few clusters. That is also not very meaningful.

How big are the clusters?

In [47]:
[len(c) for c in clusters]

[5666, 1, 1, 1, 1, 1, 1, 1, 1]

Yet an other hallmark of bad clustering: there is one big cluster.

## Cluster evaluation

We just arrived at a few quality checks on a clustering: not too few, not too many, and not too large clusters.

### Criteria
Let's formalize what a good clustering is in terms of a few thresholds.

These threshold restrain the number of clusters between a minimun and a maximum,
and demand that the largest cluster is smaller than a certain size.

All these thresholds are expressed in fractions of the total number of objects.

In [48]:
N_MIN = 0.02 # at least 2% of amount of objects
N_MAX = 0.4  # at most 40% of amount of objects
SIZE_MAX = 0.7 # largest cluster at most 70% of the objects

In [49]:
def checkQuality(clusters, nObjects):
    nC = len(clusters)
    amountClusters = nC / nObjects
    largestCluster = max(len(cluster) for cluster in clusters)
    sizeCluster = largestCluster / nObjects

    flaws = {}
    msgs = {}
    if amountClusters < N_MIN:
        msgs['amount'] = f'< {N_MIN:>4.2f}'
        flaws['amount'] = 'TOO FEW'
    elif amountClusters > N_MAX:
        msgs['amount'] = f'> {N_MAX:>4.2f}'
        flaws['amount'] = 'TOO MANY'
    else:
        msgs['amount'] = f'>= {N_MIN:>4.2f} and <= {N_MAX:>4.2f}'

    if sizeCluster > SIZE_MAX:
        msgs['size'] = f'> {SIZE_MAX:>4.2f}'
        flaws['size'] = 'TOO LARGE'
    else:
        msgs['size'] = f'<= {SIZE_MAX:>4.2f}'
        
    info1 = f'{nC:>4} clusters for {nObjects:>4} objects'
    info2 = f'largest cluster has size {largestCluster:>4}'
    msg1 = f'{amountClusters:>4.2f} {msgs["amount"]}'
    msg2 = f'{sizeCluster:>4.2f} {msgs["size"]}'
    flaw1 = f'{flaws.get("amount", "OK"):<10}'
    flaw2 = f'{flaws.get("size", "OK"):<10}'
    
    print(
        f'{flaw1}{info1:<40} relatively {msg1}\n'
        f'{flaw2}{info2:<40} relatively {msg2}'
    )

    return not flaws    

In [50]:
checkQuality(clusters, len(objects))

TOO FEW      9 clusters for 5674 objects           relatively 0.00 < 0.02
TOO LARGE largest cluster has size 5666            relatively 1.00 > 0.70


False

### Threshold search

We now run the clustering with a series of thresholds and evaluate the outcomes.

If the resulting clustering does not pass the quality check, we discard it.
Otherwise we retain it for later inspection.

We show the statistics of each clustering (good or bad), and print
the good clusterings to file.

When we write clusterings, we leave out the singleton clusters and clusters that
have half of the objects or more.

In [56]:
def experiment(objects, threshold):
    clusters = clustering(objects, threshold)
    if checkQuality(clusters, len(objects)):
        return clusters
    return False

def writeClustering(clusterings, signsFromObject, nodeType, idFeature, threshold):
    clusterFile = f'{nodeType}-{threshold}.tsv'
    clusterPath = f'{RESULT_DIR}/{clusterFile}'
    with open(clusterPath, 'w') as fh:
        fh.write(f'cluster\t{nodeType}\tobjectId\tsigns\n')
        for (i, cluster) in enumerate(sorted(
            clusterings[threshold],
            key=lambda x: -len(x),
        )):
            if len(cluster) == 1 or len(cluster) >= 0.5 * len(signsFromObject):
                continue
            for obj in cluster:
                objId = Fs(idFeature).v(obj)
                signs = ' '.join(sorted(signsFromObject[obj]))
                fh.write(f'{i}\t{obj}\t{objId}\t{signs}\n')
    display(Markdown(f'[{clusterFile}]({RESULT_GH}/{clusterFile})\n\n'))
    
    
def experiments(nodeType, idFeature, thresholds):
    (signsFromObject, objectsFromSign, emptyObjects) = getSigns(nodeType)
    objects = sorted(signsFromObject)
    
    clusterings = {}
    for threshold in thresholds:
        display(Markdown(f'**Experiment** {len(objects)} {nodeType}s *threshold* {threshold}'))
        clusters = experiment(objects, threshold)
    if clusters:
        clusterings[threshold] = clusters
        writeClustering(clusterings, signsFromObject, nodeType, idFeature, threshold)

    print(f'Found {len(clusterings)} good clusterings')
    
    return clusterings

In [57]:
experiments('tablet', 'catalogId', [0.35])
print('Done')

computed 1526 distinct sign representations from 5674 tablets (722 empty tablets)


**Experiment** 5674 tablets *threshold* 0.35

clustering 5674 objects with threshold 0.35
	1522 objects -   1000000 x memoSim - 153327270 lookups  16090051 comps
	2174 objects -   2000000 x memoSim - 154327270 lookups  16090051 comps
	2635 objects -   3000000 x memoSim - 155327270 lookups  16090051 comps
	3085 objects -   4000000 x memoSim - 156327270 lookups  16090051 comps
	3423 objects -   5000000 x memoSim - 157327270 lookups  16090051 comps
	3725 objects -   6000000 x memoSim - 158327270 lookups  16090051 comps
	4018 objects -   7000000 x memoSim - 159327270 lookups  16090051 comps
	4324 objects -   8000000 x memoSim - 160327270 lookups  16090051 comps
	4602 objects -   9000000 x memoSim - 161327270 lookups  16090051 comps
	4902 objects -  10000000 x memoSim - 162327270 lookups  16090051 comps
	5184 objects -  11000000 x memoSim - 163327270 lookups  16090051 comps
	5453 objects -  12000000 x memoSim - 164327270 lookups  16090051 comps
	5674 objects -  12887990 x memoSim - 165215260 lookups  16090051 comps
2091 clusters with t

[tablet-0.35.tsv](https://github.com/Dans-labs/Nino-cunei/blob/master/reports/clustering/tablet-0.35.tsv)



Found 1 good clusterings
Done
